# Keras RTPP

In [1]:
%matplotlib inline

In [59]:
import theano
theano.config.openmp = True
theano.config.exception_verbosity = 'low'
# theano.config.optimizer = 'fast_compile'

In [3]:
import keras

In [4]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [463]:
import multiprocessing as MP

In [5]:
sns.set(style='ticks', palette='Set2')
sns.despine()

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
from keras.layers.core import Activation, Dense

Using Theano backend.
/home/utkarshu/miniconda3/envs/py35/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


# SO

In [158]:
with open('event-train.txt', 'r') as in_file:
    eventTrain = [[int(y) for y in x.strip().split()] for x in in_file]

with open('event-test.txt', 'r') as in_file:
    eventTest = [[int(y) for y in x.strip().split()] for x in in_file]

with open('time-train.txt', 'r') as in_file:
    timeTrain = [[float(y) for y in x.strip().split()] for x in in_file]

with open('time-test.txt', 'r') as in_file:
    timeTest = [[float(y) for y in x.strip().split()] for x in in_file]

assert len(timeTrain) == len(eventTrain)
assert len(eventTest) == len(timeTest)

In [9]:
sum([len(x) for x in eventTrain])

383181

In [10]:
# Have to convert event-train data to (nb_samples, timesteps, output_dim) size

In [278]:
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Graph
from keras.layers.core import Reshape, Flatten
from keras.regularizers import l2

## Settings

In [474]:
HIDDEN_LAYER_SIZE = 128 # 64, 128, 256, 512, 1024
BATCH_SIZE = 16 # 16, 32, 64
LEARNING_RATE = 0.1 # 0.1, 0.01, 0.001
MOMENTUM = 0.9
L2_PENALTY = 0.001
EMBED_SIZE = 100 # ??

## Data transformation

In [216]:
import itertools

In [491]:
nb_samples = len(eventTrain)
max_seqlen = max(len(x) for x in eventTrain)
unique_samples = set()
for x in eventTrain + eventTest:
    unique_samples = unique_samples.union(x)
    
maxTime = max(itertools.chain((max(x) for x in timeTrain), (max(x) for x in timeTest)))
minTime = min(itertools.chain((min(x) for x in timeTrain), (min(x) for x in timeTest)))
# minTime, maxTime = 0, 1

eventTrainIn = [x[:-1] for x in eventTrain]
eventTrainOut = [x[1:] for x in eventTrain]
timeTrainIn = [[(y - minTime) / (maxTime - minTime) for y in x[:-1]] for x in timeTrain]
timeTrainOut = [[(y - minTime) / (maxTime - minTime) for y in x[1:]] for x in timeTrain]

train_event_in_seq = pad_sequences(eventTrainIn)
train_event_out_seq = pad_sequences(eventTrainOut)
train_time_in_seq = pad_sequences(timeTrainIn, dtype=float)
train_time_out_seq = pad_sequences(timeTrainOut, dtype=float)

eventTestIn = [x[:-1] for x in eventTest]
eventTestOut = [x[1:] for x in eventTest]
timeTestIn = [[(y - minTime) / (maxTime - minTime) for y in x[:-1]] for x in timeTest]
timeTestOut = [[(y - minTime) / (maxTime - minTime) for y in x[1:]] for x in timeTest]

test_event_in_seq = pad_sequences(eventTestIn)
test_event_out_seq = pad_sequences(eventTestOut)
test_time_in_seq = pad_sequences(timeTestIn, dtype=float)
test_time_out_seq = pad_sequences(timeTestOut, dtype=float)


In [492]:
nb_events = len(unique_samples)

train_event_out_hot_seq = np.zeros((nb_samples, max_seqlen - 1, nb_events), dtype=int)

for ii, evs in enumerate(eventTrainOut):
    for jj, x in enumerate(evs):
        train_event_out_hot_seq[ii, jj, x - 1] = 1
        
nb_tests = len(eventTest)

max_test_seqlen = max(len(x) for x in eventTest)
test_event_out_hot_seq = np.zeros((nb_tests, max_test_seqlen - 1, nb_events), dtype=int)

for ii, evs in enumerate(eventTestOut):
    for jj, x in enumerate(evs):
        test_event_out_hot_seq[ii, jj, x - 1] = 1

In [493]:
assert np.sum(test_event_out_hot_seq) == sum(len(x) for x in eventTestOut)
assert np.sum(train_event_out_hot_seq) == sum(len(x) for x in eventTrainOut)

In [483]:
train_event_out_hot_seq.shape

(5307, 719, 22)

In [484]:
np_utils.to_categorical(train_event_out_seq).shape

(5307, 23)

## Testing for RNN regression

In [265]:
times_in_raw = [[1,2,0], [3,1,0.1,6]]
times_in = pad_sequences(times_in_raw, dtype=float)
times_out = times_in
test_mdl = Graph()
test_mdl.add_input(name='time_input', input_shape=(None, 1))
test_mdl.add_node(SimpleRNN(HIDDEN_LAYER_SIZE, return_sequences=True), name='hidden', input='time_input')
test_mdl.add_node(Activation('relu'), input='hidden', name='rectified_hidden')
test_mdl.add_node(TimeDistributedDense(1), name='time_dense', input='rectified_hidden')
test_mdl.add_output(name='time_output', input='time_dense')

In [266]:
test_mdl.summary()

--------------------------------------------------------------------------------
Layer (name)                  Output Shape                  Param #             
--------------------------------------------------------------------------------
Layer (time_input)            (None, None, 1)               0                   
SimpleRNN (hidden)            (None, None, 64)              4224                
Activation (rectified_hidden) (None, None, 64)              0                   
TimeDistributedDense (time_den(None, None, 1)               65                  
TimeDistributedDense (time_out(None, None, 1)               65                  
--------------------------------------------------------------------------------
Total params: 4289
--------------------------------------------------------------------------------


In [267]:
%%time
test_mdl.compile('SGD', {'time_output': 'mse'})

CPU times: user 7.08 s, sys: 43.3 ms, total: 7.12 s
Wall time: 7.13 s


In [268]:
test_mdl.fit({'time_input': times_in[:,:,None], 'time_output': times_out[:,:,None]}, nb_epoch=10)

Epoch 1/10
2/2 [==============================] - 0s - loss: 5.5416
Epoch 2/10
2/2 [==============================] - 0s - loss: 4.3134
Epoch 3/10
2/2 [==============================] - 0s - loss: 3.8231
Epoch 4/10
2/2 [==============================] - 0s - loss: 3.5964
Epoch 5/10
2/2 [==============================] - 0s - loss: 3.4647
Epoch 6/10
2/2 [==============================] - 0s - loss: 3.3681
Epoch 7/10
2/2 [==============================] - 0s - loss: 3.2851
Epoch 8/10
2/2 [==============================] - 0s - loss: 3.2081
Epoch 9/10
2/2 [==============================] - 0s - loss: 3.1344
Epoch 10/10
2/2 [==============================] - 0s - loss: 3.0629


In [262]:
test_mdl.predict({ 'time_input': times_in[:,:,None] })

{'time_output': array([[[ 1.60988939],
         [ 2.02104974],
         [ 1.56695032],
         [ 2.39780569]],
 
        [[ 2.25615525],
         [ 1.83155012],
         [ 1.56925201],
         [ 2.70618749]]])}

## Model creation

Need to use an Embedding layer to represent input with variable timesteps

In [494]:
so_model = Graph()
so_model.add_input(name='event_input', input_shape=(None,), dtype=int)
so_model.add_input(name='time_input', input_shape=(None, 1))

so_model.add_node(Embedding(nb_events + 1, EMBED_SIZE, mask_zero=True),
               name='event_embedding', input='event_input')
so_model.add_node(TimeDistributedDense(1), input='time_input', name='time_scaled')
so_model.add_node(SimpleRNN(HIDDEN_LAYER_SIZE, return_sequences=True, input_shape=(None, HIDDEN_LAYER_SIZE)), 
                  name='hidden',
                  inputs=['event_embedding', 'time_scaled'])
so_model.add_node(Activation('relu'), input='hidden', name='rectified_hidden')
so_model.add_node(TimeDistributedDense(nb_events, W_regularizer=l2(L2_PENALTY)), 
                  input='rectified_hidden', 
                  name='event_output_dense')
so_model.add_node(Activation('softmax'), input='event_output_dense', name='event_softmax')
so_model.add_node(TimeDistributedDense(1, W_regularizer=l2(L2_PENALTY)),
                  input='rectified_hidden', 
                  name='time_output_dense')

so_model.add_output(name='time_output', input='time_output_dense')
so_model.add_output(name='event_output', input='event_softmax')

In [495]:
# so_model.get_config()

In [496]:
so_model.summary()

--------------------------------------------------------------------------------
Layer (name)                  Output Shape                  Param #             
--------------------------------------------------------------------------------
Layer (event_input)           (None, None)                  0                   
Layer (time_input)            (None, None, 1)               0                   
Embedding (event_embedding)   (None, None, 100)             2300                
TimeDistributedDense (time_sca(None, None, 1)               2                   
SimpleRNN (hidden)            (None, None, 128)             29440               
Activation (rectified_hidden) (None, None, 128)             0                   
TimeDistributedDense (event_ou(None, None, 22)              2838                
Activation (event_softmax)    (None, None, 22)              0                   
TimeDistributedDense (time_out(None, None, 1)               129                 
TimeDistributedDense (time_o

### Model compilation

In [497]:
%%time
so_model.compile(keras.optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM), # 'SGD'
                 {'event_output': 'categorical_crossentropy', 
                  'time_output': 'hinge'})

CPU times: user 20.2 s, sys: 357 ms, total: 20.5 s
Wall time: 20.6 s


## Model fitting

### Custom loss function

In [498]:
%%time
# so_model.compile('SGD', 
#                 {'event_output': 'categorical_crossentropy', 
#                  'time_output': lambda x,y: K.mean(keras.objectives.MSE(x, y))})

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


In [499]:
%%time
so_model.fit({
        'event_input': train_event_in_seq, 
        'event_output': train_event_out_hot_seq,
        'time_input': train_time_in_seq[:,:,None],
        'time_output': train_time_out_seq[:,:,None]
        },
        verbose=1, 
        nb_epoch=50, 
        batch_size=BATCH_SIZE)

Epoch 1/50
5307/5307 [==============================] - 137s - loss: 1.1170   
Epoch 2/50
5307/5307 [==============================] - 135s - loss: 1.0966   
Epoch 3/50
5307/5307 [==============================] - 136s - loss: 1.0921   
Epoch 4/50
5307/5307 [==============================] - 135s - loss: 1.0907   
Epoch 5/50
5307/5307 [==============================] - 135s - loss: 1.0902   
Epoch 6/50
5307/5307 [==============================] - 135s - loss: 1.0901   
Epoch 7/50
5307/5307 [==============================] - 135s - loss: 1.0898   
Epoch 8/50
5307/5307 [==============================] - 133s - loss: 1.0898   
Epoch 9/50
5307/5307 [==============================] - 132s - loss: 1.0898   
Epoch 10/50
5307/5307 [==============================] - 132s - loss: 1.0899   
Epoch 11/50
5307/5307 [==============================] - 131s - loss: 1.0894   
Epoch 12/50
5307/5307 [==============================] - 130s - loss: 1.0895   
Epoch 13/50
5307/5307 [==========================

### Saving model results to disk

In [500]:
import seqfile
so_model.save_weights(seqfile.findNextFile(prefix='so-model-', suffix='.hdf5'), overwrite=True)

## Predictions

In [531]:
def classCount(one_hot, ground_truth=None):
    if ground_truth is None:
        ground_truth = one_hot
        
    classes = np.zeros(one_hot.shape[2], dtype=int)
    for ii in range(one_hot.shape[0]):
        for jj in range(one_hot.shape[1]):
            # If the 'ground_truth' has an element, count it in one_hot
            if np.sum(ground_truth[ii, jj, :]) > 0:
                classes[np.argmax(one_hot[ii, jj, :])] += 1
    
    return classes
    

## On test data

In [502]:
%%time
pred_test = so_model.predict({ 'event_input': test_event_in_seq, 'time_input': test_time_in_seq[:, :, None] })

CPU times: user 5min 48s, sys: 7.39 s, total: 5min 55s
Wall time: 7.92 s


In [503]:
test_marker_accuracy = np.zeros(pred_test['event_output'].shape[:2], dtype=int)

In [504]:
op = pred_test['event_output']
for ii in range(op.shape[0]):
    for jj in range(op.shape[1]):
        test_marker_accuracy[ii, jj] = 1 if np.argmax(test_event_out_hot_seq[ii, jj, :]) == np.argmax(op[ii, jj, :]) else 0

Total number of test events

In [548]:
sum(len(x) - 1 for x in eventTest), np.sum(test_event_out_hot_seq)

(95907, 95907)

In [549]:
assert np.sum(test_event_out_hot_seq) == sum(len(x) - 1 for x in eventTest)

In [508]:
np.sum(test_marker_accuracy) / np.sum(test_event_out_hot_seq)

0.42893636543735075

## On training data

In [509]:
%%time
pred_train = so_model.predict({ 'event_input': train_event_in_seq, 'time_input': train_time_in_seq[:,:,None] })

CPU times: user 22min 41s, sys: 28.9 s, total: 23min 10s
Wall time: 29.9 s


In [510]:
train_marker_accuracy = np.zeros(pred_train['event_output'].shape[:2], dtype=int)

In [511]:
op = pred_train['event_output']
for ii in range(op.shape[0]):
    for jj in range(op.shape[1]):
        train_marker_accuracy[ii, jj] = 1 if np.argmax(train_event_out_hot_seq[ii, jj, :]) == np.argmax(op[ii, jj, :]) else 0

In [512]:
np.sum(train_marker_accuracy) / np.sum(train_event_out_hot_seq)

0.43463694247288781

In [535]:
%%time
# No one uses single core anymore.
# true_train_classes_old = classCount(train_event_out_hot_seq)
# true_test_classes_old = classCount(test_event_out_hot_seq)
# pred_train_classes_old = classCount(pred_train['event_output'], train_event_out_hot_seq)
# pred_test_classes_old = classCount(pred_test['event_output'], test_event_out_hot_seq)

CPU times: user 55.3 s, sys: 533 ms, total: 55.8 s
Wall time: 55.1 s


In [534]:
%%time
def worker(params):
    x, y = params
    return classCount(x, y)

with MP.Pool() as pool:
    [true_train_classes, true_test_classes, pred_train_classes, pred_test_classes] = \
        pool.map(worker,     [(train_event_out_hot_seq, train_event_out_hot_seq),
                              (test_event_out_hot_seq, test_event_out_hot_seq),
                              (pred_train['event_output'], train_event_out_hot_seq),
                              (pred_test['event_output'], test_event_out_hot_seq)])


CPU times: user 2.44 s, sys: 5.64 s, total: 8.08 s
Wall time: 31 s


In [538]:
true_train_classes

array([ 23612,   7544,   2478, 164238,  22021,  23258,   7409,   7216,
        87108,   5248,    703,  10165,   3245,   7268,   1180,    376,
         1066,   2032,   1355,    276,     56,     20])

In [539]:
np.sum(true_train_classes)

377874

In [540]:
np.sum(pred_train_classes)

377874

In [541]:
true_test_classes

array([ 6014,  2057,   689, 41138,  5627,  6208,  1949,  1785, 22046,
        1218,   164,  2557,   801,  1967,   312,   102,   272,   565,
         353,    58,    21,     4])

In [542]:
pred_test_classes

array([    0,     0,     0, 95907,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0])

In [546]:
# sns.barplot([true_train_classes, pred_train_classes]);

In [543]:
pred_train_time = pred_train['time_output'].squeeze()
pred_test_time = pred_test['time_output'].squeeze()

In [544]:
np.min(pred_train_time[0])

8.8049802780151367

In [545]:
np.max(train_time_out_seq[0])

0.9759224533138966